In [12]:
import pandas as pd
df = pd.read_csv("../data/interim/data_cleaning.csv")

/var/folders/rq/xsj46x_s2rg87wdksm1_jl3c0000gn/T/ipykernel_11086/292270290.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/interim/data_cleaning.csv")


In [14]:
# nettoyage moche du texte
# sinon à gérer avant !

import re


def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses ou crochets
    texte = re.sub(r"\([^)]*\)", "", texte)  # (...)
    # texte = re.sub(r"\[[^\]]*\]", "", texte)     # [...] # TODO: vérif avant
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["Texte_clean"] = df["Texte"].apply(nettoyer_texte)

In [20]:
import re

# Regex du champ lexical République (simplifié ici)

# sinon :
pattern_lexical = re.compile(
    # r"(?<!\w)(anti-?|pré-?|post-?|pro?)?(république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b",
    r"(république(s)?|républicain(e)?s?|républicanisme(s)?|républicanis(er|ation))",
    re.I,
)
# au pire virer le (?<!\w)(anti-?|pré-?|post-?|pro?)?
# genre on raterait irrépublicain
# aussi virer finalement la fin de mot (ce qui permettrait de matcher les s etc. sans se casser les pieds)


# Regex des expressions à exclure
pattern_excl = re.compile(
    r"\b(L|l)es Républicains\b|(P|p)résident de la République"
)  # ici garder la casse


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False


# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte_clean"].apply(contains_lexical_outside_excl)

In [21]:
df[df["repu_match_valide"]].shape

(4409, 61)

In [ ]:
mon_texte = "les Républicains n'aiment pas la République"
contains_lexical_outside_excl(mon_texte)

True

In [ ]:
mon_texte = "présidence de la République"
contains_lexical_outside_excl(mon_texte)

True

In [ ]:
mon_texte = "valeurs républicaines"
contains_lexical_outside_excl(mon_texte)

True

In [ ]:
import pandas as pd
df_full = pd.read_csv("../data/interim/extract_16.csv")

/var/folders/rq/xsj46x_s2rg87wdksm1_jl3c0000gn/T/ipykernel_11086/1120758819.py:2: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("../data/interim/extract_16.csv")


In [ ]:
df_chelou = df_full[df_full["ID_orateur"]<0]

In [ ]:
df_chelou.head()

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction
7117,CRSANR5L16S2024O1N063,RUANR5L16S2024IDS27878,SCR5A2024O1,20231127160000000,lundi 27 novembre 2023,Unique,63,AN,16,Session ordinaire 2023-2024,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Yvon Serenus,président du Collectif des ouvriers agricoles ...,-125389.0,609.79,"Au nom de notre collectif, je salue les partic...",point
7119,CRSANR5L16S2024O1N063,RUANR5L16S2024IDS27878,SCR5A2024O1,20231127160000000,lundi 27 novembre 2023,Unique,63,AN,16,Session ordinaire 2023-2024,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Philippe Pierre-Charles,porte-parole du Collectif pour dépolluer la Ma...,-125399.0,1091.66,Je remercie Marcellin Nadeau et son groupe pol...,point
7121,CRSANR5L16S2024O1N063,RUANR5L16S2024IDS27878,SCR5A2024O1,20231127160000000,lundi 27 novembre 2023,Unique,63,AN,16,Session ordinaire 2023-2024,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Janmari Flower,président de l’association Vivre Guadeloupe,-125409.0,1456.39,Je souhaite d’abord remercier le groupe GDR-NU...,point
7125,CRSANR5L16S2024O1N063,RUANR5L16S2024IDS27878,SCR5A2024O1,20231127160000000,lundi 27 novembre 2023,Unique,63,AN,16,Session ordinaire 2023-2024,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Yvon Serenus,NaN,-125389.0,1911.25,"Comme tout le monde le sait, y compris les exp...",point
7127,CRSANR5L16S2024O1N063,RUANR5L16S2024IDS27878,SCR5A2024O1,20231127160000000,lundi 27 novembre 2023,Unique,63,AN,16,Session ordinaire 2023-2024,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,M. Philippe Pierre-Charles,NaN,-125399.0,2146.85,Il faut d’abord prendre conscience des risques...,point


In [ ]:
Conséquences de la loi « immigration » sur les enfants étrangers placés à l’aide sociale à l’enfance

In [22]:
import re

# Regex du champ lexical République (simplifié ici)

# sinon :
pattern_lexical = re.compile(
    # r"(?<!\w)(anti-?|pré-?|post-?|pro?)?(république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b",
    r"républi",
    re.I,
)
# au pire virer le (?<!\w)(anti-?|pré-?|post-?|pro?)?
# genre on raterait irrépublicain
# aussi virer finalement la fin de mot (ce qui permettrait de matcher les s etc. sans se casser les pieds)


# Regex des expressions à exclure
pattern_excl = re.compile(
    r"\b(L|l)es Républicains\b|(P|p)résident de la République"
)  # ici garder la casse


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False


# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte_clean"].apply(contains_lexical_outside_excl)

In [23]:
df[df["repu_match_valide"]].shape

(4409, 61)

In [ ]:
mon_texte = "ceci est la république"
contains_lexical_outside_excl(mon_texte)


True